## Import and Load data file

In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

Using TensorFlow backend.


## Pre-process data

Here we iterate through the patterns and tokenize the sentence using nltk.word_tokenize() function and append each word in the words list. We also create a list of classes for our tags.

In [2]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

lemmatize each word and remove duplicate words from the list

In [3]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


## Create training and testing data

In [4]:

# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


## Create Model

In [5]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
47/47 [==============================] - 3s 55ms/step - loss: 2.2930 - accuracy: 0.0426
Epoch 2/200
47/47 [==============================] - 0s 1ms/step - loss: 2.2131 - accuracy: 0.1489
Epoch 3/200
47/47 [==============================] - 0s 2ms/step - loss: 2.0542 - accuracy: 0.2553
Epoch 4/200
47/47 [==============================] - 0s 1ms/step - loss: 2.0070 - accuracy: 0.4468
Epoch 5/200
47/47 [==============================] - 0s 1ms/step - loss: 1.9981 - accuracy: 0.3191
Epoch 6/200
47/47 [==============================] - 0s 1ms/step - loss: 1.7170 - accuracy: 0.5745
Epoch 7/200
47/47 [==============================] - 0s 1ms/step - loss: 1.7373 - accuracy: 0.4681
Epoch 8/200
47/47 [==============================] - 0s 1ms/step - loss: 1.5727 - accuracy: 0.5319
Epoch 9/200
47/47 [==============================] - 0s 1ms/step - loss: 1.4756 - accuracy: 0.6170
Epoch 10/200
47/47 [==============================] - 0s 1ms/step - loss: 1.3163 - accuracy: 0.5532
Epoch 11

47/47 [==============================] - 0s 1ms/step - loss: 0.0371 - accuracy: 1.0000
Epoch 84/200
47/47 [==============================] - 0s 1ms/step - loss: 0.0370 - accuracy: 1.0000
Epoch 85/200
47/47 [==============================] - 0s 1ms/step - loss: 0.0199 - accuracy: 1.0000
Epoch 86/200
47/47 [==============================] - 0s 2ms/step - loss: 0.1027 - accuracy: 0.9574
Epoch 87/200
47/47 [==============================] - 0s 1ms/step - loss: 0.1563 - accuracy: 0.9574
Epoch 88/200
47/47 [==============================] - 0s 2ms/step - loss: 0.0886 - accuracy: 0.9574
Epoch 89/200
47/47 [==============================] - 0s 2ms/step - loss: 0.0409 - accuracy: 1.0000
Epoch 90/200
47/47 [==============================] - 0s 1ms/step - loss: 0.0352 - accuracy: 1.0000
Epoch 91/200
47/47 [==============================] - 0s 1ms/step - loss: 0.0810 - accuracy: 0.9574
Epoch 92/200
47/47 [==============================] - 0s 1ms/step - loss: 0.0730 - accuracy: 0.9574
Epoch 93/200


47/47 [==============================] - 0s 2ms/step - loss: 0.0283 - accuracy: 0.9787
Epoch 165/200
47/47 [==============================] - 0s 1ms/step - loss: 0.0102 - accuracy: 1.0000
Epoch 166/200
47/47 [==============================] - 0s 1ms/step - loss: 0.0060 - accuracy: 1.0000
Epoch 167/200
47/47 [==============================] - 0s 1ms/step - loss: 0.0111 - accuracy: 1.0000
Epoch 168/200
47/47 [==============================] - 0s 1ms/step - loss: 0.0068 - accuracy: 1.0000
Epoch 169/200
47/47 [==============================] - 0s 1ms/step - loss: 0.0131 - accuracy: 1.0000
Epoch 170/200
47/47 [==============================] - 0s 1ms/step - loss: 0.0090 - accuracy: 1.0000
Epoch 171/200
47/47 [==============================] - 0s 1ms/step - loss: 0.0096 - accuracy: 1.0000
Epoch 172/200
47/47 [==============================] - 0s 1ms/step - loss: 0.0045 - accuracy: 1.0000
Epoch 173/200
47/47 [==============================] - 0s 976us/step - loss: 0.0062 - accuracy: 1.0000
Ep